In [1]:
!pip install -q transformers datasets librosa evaluate jiwer gradio bitsandbytes==0.37 accelerate rouge-score
!pip install -q git+https://github.com/huggingface/peft.git@main

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.3/76.3 MB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 29.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 MB 20.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.8/319.8 kB 20.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.7/94.7 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 436.6/436.6 kB 24.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 12.3 MB/s eta 0:00:00
   ━━━

In [2]:
# Structred Data
import numpy as np
import pandas as pd
import torch
import pickle

# System Libraries
import os
import librosa
import soundfile as sf
import nltk

# Visualization
from IPython.display import Audio, display
import matplotlib.pyplot as plt

# Whisper Model
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline
from datasets import load_dataset

# Evaluation
from rouge_score import rouge_scorer

In [3]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32
print(f"Using device: {device}")

Using device: cuda:0


# Model Loading

In [4]:
task = "transcribe"
language = "English"
language_abbr = "en" # Short hand code for the language we want to fine-tune

In [5]:
#Load Whisper Model
model_id = "openai/whisper-large-v3-turbo"

from transformers import WhisperForConditionalGeneration

model = WhisperForConditionalGeneration.from_pretrained(model_id, load_in_8bit=False, device_map="auto") #Change load_in_8bit to true if using GPU
print(model)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:90: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.26k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.62G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/3.77k [00:00<?, ?B/s]

WhisperForConditionalGeneration(
  (model): WhisperModel(
    (encoder): WhisperEncoder(
      (conv1): Conv1d(128, 1280, kernel_size=(3,), stride=(1,), padding=(1,))
      (conv2): Conv1d(1280, 1280, kernel_size=(3,), stride=(2,), padding=(1,))
      (embed_positions): Embedding(1500, 1280)
      (layers): ModuleList(
        (0-31): 32 x WhisperEncoderLayer(
          (self_attn): WhisperSdpaAttention(
            (k_proj): Linear(in_features=1280, out_features=1280, bias=False)
            (v_proj): Linear(in_features=1280, out_features=1280, bias=True)
            (q_proj): Linear(in_features=1280, out_features=1280, bias=True)
            (out_proj): Linear(in_features=1280, out_features=1280, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=1280, out_features=5120, bias=True)
          (fc2): Linear(in_features=5120, out_features=1280, bia

In [6]:
from transformers import WhisperFeatureExtractor

feature_extractor = WhisperFeatureExtractor.from_pretrained(model_id)

preprocessor_config.json:   0%|          | 0.00/340 [00:00<?, ?B/s]

In [7]:
from transformers import WhisperProcessor

processor = WhisperProcessor.from_pretrained(model_id, language="english", task="transcribe")

tokenizer_config.json:   0%|          | 0.00/283k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.71M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/494k [00:00<?, ?B/s]

normalizer.json:   0%|          | 0.00/52.7k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/34.6k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.19k [00:00<?, ?B/s]

In [8]:
from transformers import WhisperTokenizer

tokenizer = WhisperTokenizer.from_pretrained(model_id, language=language, task=task)

In [9]:
from huggingface_hub import notebook_login

notebook_login()

# Data Loading

In [9]:
def remove_columns_fn(example):
    # Manually remove the columns from the example
    return {key: value for key, value in example.items() if key not in ["accent", "age", "client_id", "down_votes", "gender", "locale", "path", "segment", "up_votes", "variant"]}


In [10]:
from datasets import load_dataset, DatasetDict, Dataset

common_voice = DatasetDict()

# Load the training and validation sets separately with streaming
train_dataset = load_dataset("mozilla-foundation/common_voice_11_0", "en", split="train", streaming=True)
validation_dataset = load_dataset("mozilla-foundation/common_voice_11_0", "en", split="validation", streaming=True)

# Combine the datasets
common_voice["train"] = train_dataset
common_voice["validation"] = validation_dataset

print(common_voice)

DatasetDict({
    train: IterableDataset({
        features: ['client_id', 'path', 'audio', 'sentence', 'up_votes', 'down_votes', 'age', 'gender', 'accent', 'locale', 'segment'],
        n_shards: 24
    })
    validation: IterableDataset({
        features: ['client_id', 'path', 'audio', 'sentence', 'up_votes', 'down_votes', 'age', 'gender', 'accent', 'locale', 'segment'],
        n_shards: 1
    })
})


In [56]:
# Apply the function to the streamed dataset
common_voice["train"] = common_voice["train"].map(remove_columns_fn)
common_voice["validation"] = common_voice["validation"].map(remove_columns_fn)

print(common_voice)

DatasetDict({
    train: IterableDataset({
        features: Unknown,
        n_shards: 24
    })
    validation: IterableDataset({
        features: Unknown,
        n_shards: 1
    })
})


In [17]:
# Create an iterator for the streaming dataset
train_dataset_iter = iter(train_dataset)


In [20]:
# Example of iterating over a few samples from the dataset
for _ in range(5):  # Fetch 5 examples
    item = next(train_dataset_iter)
    audio_data = item['audio']
    sentence = item['sentence']

    print(f"Audio data: {audio_data}")
    print(f"Sentence: {sentence}")

Audio data: {'path': 'en_train_0/common_voice_en_22788896.mp3', 'array': array([0.00000000e+00, 1.01918430e-12, 1.15186311e-12, ...,
       5.29247518e-06, 3.44609452e-06, 2.85642500e-06]), 'sampling_rate': 48000}
Sentence: The local broad is Belaugh Broad.
Audio data: {'path': 'en_train_0/common_voice_en_22788897.mp3', 'array': array([ 0.00000000e+00, -5.77955218e-13, -1.22665360e-13, ...,
       -1.70348594e-05, -1.11098052e-05, -2.50096377e-06]), 'sampling_rate': 48000}
Sentence: He also served as a trustee of the London Mosque Fund until his death.
Audio data: {'path': 'en_train_0/common_voice_en_23837344.mp3', 'array': array([ 0.00000000e+00, -5.19233745e-13, -3.00217587e-13, ...,
        2.17371075e-06, -1.20993700e-05, -1.62045199e-05]), 'sampling_rate': 48000}
Sentence: It is the second-smallest county in Montana by area.
Audio data: {'path': 'en_train_0/common_voice_en_23837345.mp3', 'array': array([ 0.00000000e+00,  1.33517478e-13,  1.49906590e-13, ...,
       -1.65660604e-05

# Alternate Way to Load Dataset

In [10]:
from datasets import load_dataset, DatasetDict, Dataset

common_voice = DatasetDict()

# Load the training and validation sets separately with streaming
train_dataset = load_dataset("mozilla-foundation/common_voice_11_0", "en", split="train", streaming=True)
validation_dataset = load_dataset("mozilla-foundation/common_voice_11_0", "en", split="validation", streaming=True)


common_voice_11_0.py:   0%|          | 0.00/8.13k [00:00<?, ?B/s]

README.md:   0%|          | 0.00/14.4k [00:00<?, ?B/s]

languages.py:   0%|          | 0.00/3.44k [00:00<?, ?B/s]

release_stats.py:   0%|          | 0.00/60.9k [00:00<?, ?B/s]

The repository for mozilla-foundation/common_voice_11_0 contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/mozilla-foundation/common_voice_11_0.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


In [11]:
train_samples = []
for sample in train_dataset:
    train_samples.append(sample)
    if len(train_samples) >= 500:
        break

# If we have less than 500, take the remaining from the validation dataset
if len(train_samples) < 500:
    for sample in validation_dataset:
        train_samples.append(sample)
        if len(train_samples) >= 500:
            break

# Create a dataset from the streamed samples
common_voice["train"] = Dataset.from_dict({k: [d[k] for d in train_samples] for k in train_samples[0]})

# Load the test set and stream 100 samples for testing
test_samples = []
for sample in load_dataset("mozilla-foundation/common_voice_11_0", "en", split="test", streaming=True):
    test_samples.append(sample)
    if len(test_samples) >= 100:
        break

# Create the test dataset
common_voice["test"] = Dataset.from_dict({k: [d[k] for d in test_samples] for k in test_samples[0]})

Reading metadata...: 948736it [00:21, 44104.86it/s]
Reading metadata...: 16354it [00:00, 19639.30it/s]


In [18]:
# Print dataset details
print("Training Dataset:", common_voice["train"])
print("Test Dataset:", common_voice["test"])

Training Dataset: Dataset({
    features: ['client_id', 'path', 'audio', 'sentence', 'up_votes', 'down_votes', 'age', 'gender', 'accent', 'locale', 'segment'],
    num_rows: 500
})
Test Dataset: Dataset({
    features: ['client_id', 'path', 'audio', 'sentence', 'up_votes', 'down_votes', 'age', 'gender', 'accent', 'locale', 'segment'],
    num_rows: 100
})


In [19]:
common_voice = common_voice.remove_columns(["accent", "age", "client_id", "down_votes", "gender", "locale", "path", "segment", "up_votes"])

# Preparing the Tuning

In [12]:
import torch

from dataclasses import dataclass
from typing import Any, Dict, List, Union


@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lengths and need different padding methods
        # first treat the audio inputs by simply returning torch tensors
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        # get the tokenized label sequences
        label_features = [{"input_ids": feature["labels"]} for feature in features]
        # pad the labels to max length
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        # if bos token is appended in previous tokenization step,
        # cut bos token here as it's append later anyways
        if (labels[:, 0] == self.processor.tokenizer.bos_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels

        return batch

In [13]:
data_collator = DataCollatorSpeechSeq2SeqWithPadding(processor=processor)

In [14]:
import evaluate

metric = evaluate.load("wer")

In [16]:
!pip install -q git+https://github.com/huggingface/peft.git@main

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [22]:
from peft import prepare_model_for_kbit_training

model = prepare_model_for_kbit_training(model, output_embedding_layer_name="proj_out")

TypeError: prepare_model_for_kbit_training() got an unexpected keyword argument 'output_embedding_layer_name'

In [ ]:
from peft import LoraConfig, PeftModel, LoraModel, LoraConfig, get_peft_model

config = LoraConfig(r=32, lora_alpha=64, target_modules=["q_proj", "v_proj"], lora_dropout=0.05, bias="none")

model = get_peft_model(model, config)
model.print_trainable_parameters()

In [ ]:
def make_inputs_require_grad(module, input, output):
    output.requires_grad_(True)

model.model.encoder.conv1.register_forward_hook(make_inputs_require_grad)

In [ ]:
from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir="realtime_asr",  # change to a repo name of your choice
    per_device_train_batch_size=8,
    gradient_accumulation_steps=1,  # increase by 2x for every 2x decrease in batch size
    learning_rate=1e-3,
    warmup_steps=50,
    num_train_epochs=1,
    evaluation_strategy="steps",
    fp16=True,
    per_device_eval_batch_size=8,
    generation_max_length=128,
    logging_steps=100,
    max_steps=100, # only for testing purposes, remove this from your final run :)
    remove_unused_columns=False,  # required as the PeftModel forward doesn't have the signature of the wrapped model's forward
    label_names=["labels"],  # same reason as above
)

# Training

In [ ]:
from transformers import Seq2SeqTrainer, TrainerCallback, TrainingArguments, TrainerState, TrainerControl
from transformers.trainer_utils import PREFIX_CHECKPOINT_DIR

# This callback helps to save only the adapter weights and remove the base model weights.
class SavePeftModelCallback(TrainerCallback):
    def on_save(
        self,
        args: TrainingArguments,
        state: TrainerState,
        control: TrainerControl,
        **kwargs,
    ):
        checkpoint_folder = os.path.join(args.output_dir, f"{PREFIX_CHECKPOINT_DIR}-{state.global_step}")

        peft_model_path = os.path.join(checkpoint_folder, "adapter_model")
        kwargs["model"].save_pretrained(peft_model_path)

        pytorch_model_path = os.path.join(checkpoint_folder, "pytorch_model.bin")
        if os.path.exists(pytorch_model_path):
            os.remove(pytorch_model_path)
        return control


trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=common_voice["train"],
    eval_dataset=common_voice["test"],
    data_collator=data_collator,
    # compute_metrics=compute_metrics,
    tokenizer=processor.feature_extractor,
    callbacks=[SavePeftModelCallback],
)
model.config.use_cache = False  # silence the warnings. Please re-enable for inference!

In [ ]:
trainer.train()